In [1]:
from sympy import *
import math
init_printing()

In [2]:
theta, l = symbols('theta l')

H = Matrix([l*cos(theta), l*sin(theta)])
Hdot = H.calc_J([theta])

Hdot

⎡-l⋅sin(θ)⎤
⎢         ⎥
⎣l⋅cos(θ) ⎦

In [3]:
Hdot.subs(
    [
        (l, 0.46),
        (theta,35*math.pi/180)
    ]
)

⎡-0.263845160721481⎤
⎢                  ⎥
⎣0.376809940372936 ⎦

In [4]:
t = symbols('t')

Hddot = Hdot.calc_J([theta])*Derivative(theta,t)
Hddot


⎡          d    ⎤
⎢-l⋅cos(θ)⋅──(θ)⎥
⎢          dt   ⎥
⎢               ⎥
⎢          d    ⎥
⎢-l⋅sin(θ)⋅──(θ)⎥
⎣          dt   ⎦

In [5]:
J = Hdot
J1 = (J.T*J)**(-1)*J.T
simplify(J1)


⎡-sin(θ)   cos(θ)⎤
⎢────────  ──────⎥
⎣   l        l   ⎦

In [6]:
simplify(J1*J)


[1]

In [7]:
g, t = symbols('g t')
m1, m2, l1, l2, r1, r2, I1, I2 = symbols('m1 m2 l1 l2 r1 r2 I1 I2')
m = Matrix([m1, m2])
l = Matrix([l1, l2])
r = Matrix([r1, r2])
I = Matrix([I1, I2])

theta_s = Function('theta_s')(t)
theta_e = Function('theta_e')(t)
theta = Matrix([theta_s, theta_e])

x1 = Matrix([r1*sin(theta_s), 
             -r1*cos(theta_s)])
x2 = Matrix([l1*sin(theta_s)+r2*sin(theta_s+theta_e),
             -l1*cos(theta_s)-r2*cos(theta_s+theta_e)])

x1d = diff(x1, t)
x2d = diff(x2, t)
thetad = diff(theta, t)

Tlinear1 = 1/2.*m1*(x1d[0]**2+x1d[1]**2) 
Tlinear2 = 1/2.*m2*(x2d[0]**2+x2d[1]**2)
Trot1 = 1/2.*I1*thetad[0]**2
Trot2 = 1/2.*I2*(thetad[0]+thetad[1])**2
T = Tlinear1 + Tlinear2 + Trot1 + Trot2

U1 = r1*m1*g*(1-cos(theta_s))
U2 = l1*m2*g*(1-cos(theta_s)) + r2*m2*g*(1-cos(theta_s+theta_e))
U = U1 + U2

L = T - U
L = simplify(L)

tau = Matrix([
    diff( diff(L, diff(theta_s)), t) - diff(L, theta_s),
    diff( diff(L, diff(theta_e)), t) - diff(L, theta_e),
])
tau = simplify(nsimplify(tau))
tau = tau.rewrite(exp).expand().powsimp().rewrite(sin).expand()

tau = Matrix([
    collect(tau[0], Derivative(Derivative(theta_s, t), t)), 
    collect(tau[1], Derivative(Derivative(theta_s, t), t))
])
tau = Matrix([
    collect(tau[0], Derivative(Derivative(theta_e, t), t)), 
    collect(tau[1], Derivative(Derivative(theta_e, t), t))
])

tau = Matrix([
    collect(tau[0], sin(theta_s)),
    collect(tau[1], sin(theta_e))
])


In [8]:
tau[0]


                                                               2              
                                                    ⎛d        ⎞               
g⋅m₂⋅r₂⋅sin(θₑ(t))⋅cos(θₛ(t)) - l₁⋅m₂⋅r₂⋅sin(θₑ(t))⋅⎜──(θₑ(t))⎟  - 2⋅l₁⋅m₂⋅r₂⋅
                                                    ⎝dt       ⎠               
                                                                              

                                                                       2      
           d         d           ⎛                                2⎞  d       
sin(θₑ(t))⋅──(θₑ(t))⋅──(θₛ(t)) + ⎝I₂ + l₁⋅m₂⋅r₂⋅cos(θₑ(t)) + m₂⋅r₂ ⎠⋅───(θₑ(t)
           dt        dt                                                2      
                                                                     dt       

                                                                              
                                                          ⎛            2      
) + (g⋅l₁⋅m₂ + g⋅m₁⋅r₁ + g⋅m₂⋅r₂⋅cos(θₑ(t)))⋅sin(θ

In [9]:
tau[1]




                                                2          ⎛                  
                                ⎛          2⎞  d           ⎜                  
g⋅m₂⋅r₂⋅sin(θₛ(t))⋅cos(θₑ(t)) + ⎝I₂ + m₂⋅r₂ ⎠⋅───(θₑ(t)) + ⎜g⋅m₂⋅r₂⋅cos(θₛ(t))
                                                2          ⎝                  
                                              dt                              

                       2⎞                                                    2
            ⎛d        ⎞ ⎟              ⎛                                2⎞  d 
 + l₁⋅m₂⋅r₂⋅⎜──(θₛ(t))⎟ ⎟⋅sin(θₑ(t)) + ⎝I₂ + l₁⋅m₂⋅r₂⋅cos(θₑ(t)) + m₂⋅r₂ ⎠⋅───
            ⎝dt       ⎠ ⎠                                                    2
                                                                           dt 

       
       
(θₛ(t))
       
       